In [3]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import importlib
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, lesioned_sim, compute_prob_tensor, init_params_tensor, num_stab_help, proxy_sim
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

<module 'granch_utils.lesioned_sim' from '/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/lesioned_sim.py'>

In [151]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)


device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 20, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 20, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 20, 
    "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 20, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 1,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 2, 
    "world_EIGs": 3, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(pair_each_stim = 1, n_feature=3)
print(tensor_stimuli)


tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan, 
                linking_hypothesis = "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

if params.linking_hypothesis == "EIG": 
    res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
else: 
    res = proxy_sim.granch_proxy_sim(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])



pd.set_option('display.max_rows', None)

print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)



[<granch_utils.init_model_tensor.granch_stimuli object at 0x2aca5d810>, <granch_utils.init_model_tensor.granch_stimuli object at 0x2aca5e830>, <granch_utils.init_model_tensor.granch_stimuli object at 0x2aca5e0e0>, <granch_utils.init_model_tensor.granch_stimuli object at 0x2aca5d570>, <granch_utils.init_model_tensor.granch_stimuli object at 0x17fcdeb60>, <granch_utils.init_model_tensor.granch_stimuli object at 0x16c65cdf0>, <granch_utils.init_model_tensor.granch_stimuli object at 0x16c65cf40>, <granch_utils.init_model_tensor.granch_stimuli object at 0x2ac8fa950>]
0
tensor([0.2150, 0.4908, 0.2151], dtype=torch.float64)
tensor(0.9208, dtype=torch.float64)
1
tensor([0.1089, 0.3765, 0.1090], dtype=torch.float64)
tensor(0.5944, dtype=torch.float64)
2
tensor([0.0609, 0.2414, 0.0610], dtype=torch.float64)
tensor(0.3633, dtype=torch.float64)
3
tensor([1.2934e-08, 1.4368e-02, 3.4781e-07], dtype=torch.float64)
tensor(0.0144, dtype=torch.float64)
4
tensor([0.0386, 0.0980, 0.0386], dtype=torch.floa

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.14589567486112076' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()
/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.02436654641184847' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()
/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureW

tensor([ 2.0311e-11,  3.6118e-08, -5.7864e-11], dtype=torch.float64)
tensor(3.6080e-08, dtype=torch.float64)
12
tensor([-1.1375e-11,  2.5099e-09, -3.5886e-12], dtype=torch.float64)
tensor(2.4950e-09, dtype=torch.float64)
13
tensor([-9.6109e-12,  1.8974e-10, -3.9728e-12], dtype=torch.float64)
tensor(1.7615e-10, dtype=torch.float64)
14
tensor([ 6.8059e-13, -3.3900e-11,  4.2007e-11], dtype=torch.float64)
tensor(8.7874e-12, dtype=torch.float64)
15
tensor([4.5901e-11, 8.8745e-11, 6.1052e-13], dtype=torch.float64)
tensor(1.3526e-10, dtype=torch.float64)
16
tensor([-3.9836e-11, -1.2085e-11, -3.8337e-11], dtype=torch.float64)
tensor(-9.0258e-11, dtype=torch.float64)
17
tensor([-2.9609e-12,  1.3544e-11, -1.0878e-11], dtype=torch.float64)
tensor(-2.9470e-13, dtype=torch.float64)
18
tensor([ 4.2824e-11, -7.9610e-11,  2.0715e-11], dtype=torch.float64)
tensor(-1.6071e-11, dtype=torch.float64)
19
tensor([ 3.9940e-11,  9.1615e-11, -2.5399e-11], dtype=torch.float64)
tensor(1.0616e-10, dtype=torch.floa

tensor(-0.1622)

# Test 

In [90]:
importlib.reload(init_model_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(proxy_sim)

test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
test_stimuli.add_toy_example(0.3, 0.7)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = 3,
                #world_EIGs = PRIOR_INFO["world_EIGs"],
                #max_observation = PRIOR_INFO["max_observation"], 
                max_observation = 500,
                forced_exposure_max= np.nan, 
                linking_hypothesis= "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

pd.set_option('display.max_rows', None)

#res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)
#res.behavior

res.behavior

0
tensor([1.6102, 1.6102, 1.6102], dtype=torch.float64)
tensor(4.8305, dtype=torch.float64)
1
tensor([ 3.2844e-10, -1.7203e-12, -1.9707e-14], dtype=torch.float64)
tensor(3.2670e-10, dtype=torch.float64)
2
tensor([-3.3853e-19, -1.5943e-27, -1.5177e-31], dtype=torch.float64)
tensor(-3.3853e-19, dtype=torch.float64)
3
tensor([-5.7233e-30, -1.5571e-43, -1.6133e-51], dtype=torch.float64)
tensor(-5.7233e-30, dtype=torch.float64)
4
tensor([-9.3345e-43, -6.9023e-58, -7.4702e-75], dtype=torch.float64)
tensor(-9.3345e-43, dtype=torch.float64)
5
tensor([ -9.3341e-54,  -4.1175e-78, -5.2217e-101], dtype=torch.float64)
tensor(-9.3341e-54, dtype=torch.float64)
6
tensor([ -4.9936e-70, -1.0278e-101, -2.5024e-116], dtype=torch.float64)
tensor(-4.9936e-70, dtype=torch.float64)
7
tensor([ -8.8213e-80, -1.1678e-116, -1.0008e-126], dtype=torch.float64)
tensor(-8.8213e-80, dtype=torch.float64)
8
tensor([-3.7244e-101, -6.8210e-126, -9.7724e-173], dtype=torch.float64)
tensor(-3.7244e-101, dtype=torch.float64)


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.19104316284168504' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()
/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.06911857673116685' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()
/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:298: FutureWa

tensor([-4.0853e-28, -1.0748e-41, -2.4303e-50], dtype=torch.float64)
tensor(-4.0853e-28, dtype=torch.float64)
23
tensor([-2.9365e-48, -2.1314e-71, -2.2213e-88], dtype=torch.float64)
tensor(-2.9365e-48, dtype=torch.float64)
24
tensor([ -1.2788e-70,  -4.5830e-98, -2.0161e-124], dtype=torch.float64)
tensor(-1.2788e-70, dtype=torch.float64)
25
tensor([ -1.3162e-90, -1.8131e-137, -1.3475e-177], dtype=torch.float64)
tensor(-1.3162e-90, dtype=torch.float64)
26
tensor([-5.4351e-117, -2.6688e-178, -3.4454e-197], dtype=torch.float64)
tensor(-5.4351e-117, dtype=torch.float64)
27
tensor([-3.7596e-139, -1.7227e-197, -3.1550e-224], dtype=torch.float64)
tensor(-3.7596e-139, dtype=torch.float64)
28
tensor([-9.6170e-178, -2.2079e-223, -6.1810e-310], dtype=torch.float64)
tensor(-9.6170e-178, dtype=torch.float64)
29
tensor([-2.2773e-185, -2.3471e-306,   0.0000e+00], dtype=torch.float64)
tensor(-2.2773e-185, dtype=torch.float64)
30
tensor([-1.2139e-219, -1.9234e-319,   0.0000e+00], dtype=torch.float64)
te

,stimulus_id,EIG,Look_away,surprisal
0,0,NaN,False,56.743639
1,0,NaN,False,47.836114
2,0,NaN,False,47.822912
3,0,NaN,False,47.827790
4,0,NaN,False,47.824661
5,0,NaN,False,47.830879
6,0,NaN,False,47.818072
7,0,NaN,False,47.826915
8,0,NaN,False,47.824374
9,0,NaN,False,47.829967


In [15]:
res.behavior

,stimulus_id,EIG,Look_away,surprisal
0,0,NaN,False,8.866722
1,0,NaN,False,5.499945
2,0,NaN,False,5.406920
3,0,NaN,False,5.393750
4,0,NaN,False,5.399595
...,...,...,...,...
495,NaN,NaN,NaN,NaN
496,NaN,NaN,NaN,NaN
497,NaN,NaN,NaN,NaN
498,NaN,NaN,NaN,NaN


RuntimeError: torch.meshgrid: Expected 0D or 1D tensor in the tensor list but got:  3  5
 3  6
 4  5
 4  6
[ torch.LongTensor{4,2} ]